In [1]:
import sys
import os
sys.path.append(os.getcwd()[:-8])

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from ACME.ACME import ACME 
import time

In [3]:
from plotly.io import write_image
import matplotlib.pyplot as plt

In [4]:
glass_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data',index_col=[0])
glass_data.columns = ['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','Type']

In [5]:
features = glass_data.drop(columns='Type').columns.to_list()

In [6]:
glass_data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
1,,,,,,,,,,
2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.00,1
3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.00,1
4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.00,1
5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.00,1
6,1.51596,12.79,3.61,1.62,72.97,0.64,8.07,0.00,0.26,1
...,...,...,...,...,...,...,...,...,...,...
210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.00,7
211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.00,7
212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.00,7


In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(glass_data, test_size=0.3, stratify=glass_data['Type'].values, random_state=1234)

In [21]:
cbc = CatBoostClassifier().fit( train[features].values, train['Type'].values )

Learning rate set to 0.071805
0:	learn: 1.7408948	total: 3.47ms	remaining: 3.46s
1:	learn: 1.6666088	total: 5.41ms	remaining: 2.7s
2:	learn: 1.6174880	total: 7.15ms	remaining: 2.38s
3:	learn: 1.5704830	total: 8.9ms	remaining: 2.22s
4:	learn: 1.5288786	total: 11.4ms	remaining: 2.26s
5:	learn: 1.4749263	total: 13.8ms	remaining: 2.28s
6:	learn: 1.4268523	total: 15.9ms	remaining: 2.26s
7:	learn: 1.3894817	total: 18.2ms	remaining: 2.25s
8:	learn: 1.3610985	total: 19.8ms	remaining: 2.18s
9:	learn: 1.3318217	total: 21.4ms	remaining: 2.11s
10:	learn: 1.2973148	total: 22.6ms	remaining: 2.03s
11:	learn: 1.2689710	total: 23.9ms	remaining: 1.97s
12:	learn: 1.2439952	total: 25.2ms	remaining: 1.91s
13:	learn: 1.2154543	total: 26.5ms	remaining: 1.86s
14:	learn: 1.1916688	total: 28.4ms	remaining: 1.86s
15:	learn: 1.1665618	total: 29.8ms	remaining: 1.83s
16:	learn: 1.1389104	total: 31.3ms	remaining: 1.81s
17:	learn: 1.1152426	total: 32.8ms	remaining: 1.79s
18:	learn: 1.0968550	total: 34ms	remaining: 1.

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(test['Type'].values, cbc.predict(test[features].values), normalize=True)

0.84375

In [36]:
acme_cb = ACME(cbc, 'Type',K=50, task = 'class')

In [37]:
start = time.time()
acme_cb = acme_cb.fit(glass_data,robust = True, label_class=None)
print(time.time()-start)

0.5799081325531006


In [25]:
import shap 
start = time.time()
shap_cb = shap.KernelExplainer(cbc.predict, glass_data[features].values)
shap_ct_values = shap_cb.shap_values(glass_data[features].values)
print(time.time()-start)

Using 213 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/213 [00:00<?, ?it/s]

194.37456011772156


In [41]:
acme_cb.summary_plot()

In [27]:
acme_cb.bar_plot()

In [28]:
acme_class =  acme_cb.fit(glass_data,robust = True, label_class=1)

In [29]:
acme_class.summary_plot()

In [30]:
local_acme = acme_class.fit_local(glass_data,local=100, label_class=1)

In [31]:
local_acme.summary_plot(local=True)